In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import time
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [17]:
df=pd.read_excel(r"../dataset/data.xlsx",sheet_name="Sheet1")

df

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350
...,...,...,...,...,...,...,...
4021,22,66,6,6.0,50,25.00,1.628000
4022,22,66,6,6.0,75,25.00,2.714000
4023,22,66,6,6.0,100,25.00,2.879000
4024,22,66,6,7.5,0,25.00,0.602680


In [18]:
df.shape

(4026, 7)

In [19]:
df.head()

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350


In [20]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df)

normalized_df = pd.DataFrame(normalized_data, columns=df.columns)


print(normalized_df)

      Number of Storeys(NS)  Height of Structure(HS)  Number of Spans(HSP)  \
0                       0.0                      0.0                   0.0   
1                       0.0                      0.0                   0.0   
2                       0.0                      0.0                   0.0   
3                       0.0                      0.0                   0.0   
4                       0.0                      0.0                   0.0   
...                     ...                      ...                   ...   
4021                    1.0                      1.0                   1.0   
4022                    1.0                      1.0                   1.0   
4023                    1.0                      1.0                   1.0   
4024                    1.0                      1.0                   1.0   
4025                    1.0                      1.0                   1.0   

      Length of Spans (LS)  Opening percentage (OP)  \
0       

In [21]:
normalized_df.keys()

Index(['Number of Storeys(NS)', 'Height of Structure(HS)',
       'Number of Spans(HSP)', 'Length of Spans (LS)',
       'Opening percentage (OP)', 'Masonry wall Stiffeness Et (MS)',
       'Period (P)'],
      dtype='object')

In [22]:
X=normalized_df.drop("Period (P)" ,axis=1)

In [23]:
y=normalized_df["Period (P)"]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=99)

# knn

In [25]:
from sklearn.model_selection import KFold,cross_validate
from bayes_opt import BayesianOptimization
from hyperopt import hp,fmin,tpe,Trials,partial
from hyperopt.early_stop import no_progress_loss
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
import optuna.integration.lightgbm as oplgb
from sklearn.neighbors import KNeighborsRegressor

In [11]:
def bayesopt_objective(n_neighbors,p):
    model=KNeighborsRegressor(n_neighbors=int(n_neighbors)
              ,p=int(p))
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                   ,error_score='raise')
    
    return np.mean(abs(validate_loss['test_score']))

In [12]:
def param_bayes_opt(init_points,n_iter):
    opt=BayesianOptimization(bayesopt_objective
                             ,param_grid_simple
                             ,random_state=99)
    
   
    opt.maximize(init_points=init_points 
                 ,n_iter=n_iter 
                )
    
       params_best=opt.max['params'] 
    score_best=opt.max['target'] 
    
        print("\n","best params: ", params_best,
          "\n","best cvscore: ", score_best)
    
    return params_best,score_best

In [26]:
def bayes_opt_validation(params_best):
    model=KNeighborsRegressor(n_neighbors=int(params_best['n_neighbors'])
            ,p=int(params_best['p'])
           )
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                  )
    
    return np.mean(abs(validate_loss['test_score']))

In [27]:
# Start time
start_time = time.time()

param_grid_simple={'n_neighbors':(3,15)
                  ,'p':(1,2)
                  }

params_best,score_best=param_bayes_opt(20,100)

params_best 
score_best 
validation_score=bayes_opt_validation(params_best) 
validation_score 
# End time
end_time = time.time()
# Calculate total time
total_time = end_time - start_time
print("Total time taken: ", total_time)

|   iter    |  target   | n_neig... |     p     |
-------------------------------------------------
| 1         | 0.02354   | 11.07     | 1.488     |
| 2         | 0.02378   | 12.91     | 1.031     |
| 3         | 0.02378   | 12.7      | 1.566     |
| 4         | 0.02234   | 6.571     | 1.047     |
| 5         | 0.02454   | 14.89     | 1.007     |
| 6         | 0.02378   | 12.24     | 1.747     |
| 7         | 0.02219   | 7.529     | 1.494     |
| 8         | 0.02454   | 14.15     | 1.395     |
| 9         | 0.02454   | 14.69     | 1.524     |
| 10        | 0.02253   | 4.123     | 1.813     |
| 11        | 0.02225   | 5.54      | 1.554     |
| 12        | 0.02234   | 6.507     | 1.816     |
| 13        | 0.02378   | 12.94     | 1.222     |
| 14        | 0.02316   | 10.74     | 1.095     |
| 15        | 0.02219   | 7.94      | 1.097     |
| 16        | 0.02253   | 4.728     | 1.212     |
| 17        | 0.02236   | 8.72      | 1.078     |
| 18        | 0.02225   | 5.821     | 1.007     |


| 101       | 0.02454   | 14.7      | 1.527     |
| 102       | 0.02454   | 14.55     | 1.259     |
| 103       | 0.02236   | 8.612     | 1.592     |
| 104       | 0.02454   | 14.39     | 1.011     |
| 105       | 0.02354   | 11.8      | 1.134     |
| 106       | 0.02454   | 14.68     | 1.526     |
| 107       | 0.02454   | 14.13     | 1.408     |
| 108       | 0.02225   | 5.562     | 1.19      |
| 109       | 0.02422   | 13.53     | 2.0       |
| 110       | 0.02219   | 7.17      | 1.821     |
| 111       | 0.02422   | 13.34     | 1.791     |
| 112       | 0.02454   | 14.69     | 1.51      |
| 113       | 0.02454   | 14.98     | 1.924     |
| 114       | 0.02454   | 14.14     | 1.388     |
| 115       | 0.02454   | 14.7      | 1.52      |
| 116       | 0.02454   | 14.98     | 1.996     |
| 117       | 0.02454   | 14.9      | 1.025     |
| 118       | 0.02219   | 7.979     | 1.463     |
| 119       | 0.02236   | 8.208     | 1.943     |
| 120       | 0.02225   | 5.146     | 1.227     |


In [28]:
from sklearn.neighbors import KNeighborsRegressor

In [29]:
knn_reg=KNeighborsRegressor(**params_best )
params_best['n_neighbors'] = int(params_best['n_neighbors'])
knn_reg.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=15)

In [30]:
y_pred= knn_reg.predict(X_test)
y_pred1=knn_reg.predict(X_val)
y_pred2=knn_reg.predict(X_train)

In [31]:
print("mean_absolute_error:", mean_absolute_error(y_val, y_pred1))
print("mean_squared_error:", mean_squared_error(y_val, y_pred1))
print("rmse:", sqrt(mean_squared_error(y_val, y_pred1)))
print("r2 score:", r2_score(y_val, y_pred1))

mean_absolute_error: 0.020397900825814438
mean_squared_error: 0.000905815201059157
rmse: 0.030096763963242908
r2 score: 0.981832394274672


In [32]:
print("mean_absolute_error:", mean_absolute_error(y_train, y_pred2))
print("mean_squared_error:", mean_squared_error(y_train, y_pred2))
print("rmse:", sqrt(mean_squared_error(y_train, y_pred2)))
print("r2 score:", r2_score(y_train, y_pred2))

mean_absolute_error: 0.01797663087579595
mean_squared_error: 0.0006974112480582599
rmse: 0.026408544981847446
r2 score: 0.9857424708552366


In [33]:
print("mean_absolute_error:", mean_absolute_error(y_test, y_pred))
print("mean_squared_error:", mean_squared_error(y_test, y_pred))
print("rmse:", sqrt(mean_squared_error(y_test, y_pred)))
print("r2 score:", r2_score(y_test, y_pred))

mean_absolute_error: 0.020453030874377297
mean_squared_error: 0.000903878038355965
rmse: 0.030064564496362908
r2 score: 0.982356698612388
